In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns

In [2]:
df = pd.read_csv("../data/NBADiamondMine.csv")

In [4]:
df.shape

(815, 31)

**Clean column headers**

In [14]:
df.columns = df.columns.str.strip()
df.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)

In [15]:
df.columns

Index(['#', 'Date', 'Away_Team', '1Q', '2Q', '3Q', '4Q', 'Total', 'Home_Team',
       '1Q.1', '2Q.1', '3Q.1', '4Q.1', 'Total.1', 'Game_Total', 'Over/Under',
       'Result', 'Favorite', 'Spread', 'Home_Pts/G', 'Home_Opp_Pts/G',
       'Home_SRS', 'Home_Pace', 'Home_Off_Rat', 'Home_Def_Rat', 'Away_Pts/G',
       'Away_Opp_Pts/G', 'Away_SRS', 'Away_Pace', 'Away_Off_Rat',
       'Away_Def_Rat'],
      dtype='object')

**Clean spreads**

In [16]:
df.Spread.unique()

array(['-4.5', '-9.5', '-3', '-6.5', '-3.5', '-1.5', '-2', '-5.5', '-2.5',
       '-1', '-7', '-13.5', '-12', '-6', '-9', '-11.5', '-10.5', '-8.5',
       '-4', '-11', '-8', '-5', '-15', '-7.5', '-12.5', '-14.5', '-10',
       'PK', '-13', '-16', '-18.5', '-18', '-14', '3.5', '7', '4.5', '1',
       '4', '6', '14', '2.5', '5', '5.5', '10.5', '7.5', '13', '15',
       '11.5', '16', '8', '10', '14.5', '11', '6.5'], dtype=object)

In [28]:
clean_spreads = []
for index, row in df.iterrows():
    if row['Spread'] == 'PK':
        row['Spread'] = 0
        clean_spreads.append(float(row['Spread']))
    elif float(row['Spread']) < 0:
        clean_spreads.append(float(row['Spread'])*-1)
    else:
        clean_spreads.append(float(row['Spread']))

In [30]:
df['Spread'] = clean_spreads

**Fix team name abbreviations such that there is consistency across different columns**

In [35]:
fixed_abbrevs = []
for index, row in df.iterrows():
    if row['Favorite'] == 'BKN':
        fixed_abbrevs.append('BRK')
    elif row['Favorite'] == 'CHA':
        fixed_abbrevs.append('CHO')
    elif row['Favorite'] == 'NOR':
        fixed_abbrevs.append('NOP')
    elif row['Favorite'] == 'UTH':
        fixed_abbrevs.append('UTA')
    else:
        fixed_abbrevs.append(row['Favorite'])

In [42]:
df['Favorite'] = fixed_abbrevs

**Binary field indicating whether the favorite was the home team or the away team**

In [40]:
df['Favorite_binary'] = np.where(df['Favorite']==df['Home_Team'], 'Home', 'Away')

**Points difference**

In [48]:
df['Points_Diff_Fav-Under'] = np.where(df['Favorite_binary']=='Home', 
                             df['Total.1'] - df['Total'], 
                             df['Total'] - df['Total.1'])

**Point differential vs Spread to d

In [54]:
df['Diff_from_Spread'] = df['Points_Diff_Fav-Under'] - df['Spread']
df['Spread_Winner'] = np.where(df['Diff_from_Spread'] > 0, 'Favorite', 'Underdog')

In [55]:
df.sample(10)

,#,Date,Away_Team,1Q,2Q,3Q,4Q,Total,Home_Team,1Q.1,...,Away_Pts/G,Away_Opp_Pts/G,Away_SRS,Away_Pace,Away_Off_Rat,Away_Def_Rat,Favorite_binary,Points_Diff_Fav-Under,Diff_from_Spread,Spread_Winner
494,496,12/25/2017,WAS,30,22,28,31,111,BOS,24,...,107.3,105.6,1.11,97.4,109.5,107.8,Home,-8,-12.0,Underdog
587,589,1/7/2018,OKC,23,30,29,18,100,PHO,28,...,106.5,102.5,3.83,95.8,110.3,106.1,Away,-14,-22.5,Underdog
593,595,1/8/2018,HOU,35,25,27,29,116,CHI,22,...,113.8,106.1,7.39,98.6,114.9,107.1,Away,9,3.0,Favorite
666,668,1/19/2018,SAS,18,19,23,23,83,TOR,17,...,100.7,97.6,2.61,94.3,106.8,103.5,Home,3,-3.0,Underdog
500,502,12/26/2017,CHI,22,34,27,32,115,MIL,31,...,103.6,109.3,-5.29,98.1,104.1,109.8,Home,-9,-16.5,Underdog
194,196,11/13/2017,ATL,32,18,30,25,105,NOP,23,...,103.6,108.3,-4.73,97.2,106.4,111.2,Away,-1,-10.5,Underdog
326,328,12/2/2017,LAC,17,21,28,16,82,DAL,24,...,107.6,107.3,0.27,98.3,109.2,108.9,Home,26,21.5,Favorite
636,638,1/14/2018,POR,33,10,31,29,103,MIN,31,...,104.9,103.1,1.33,96.4,108.4,106.4,Home,17,11.0,Favorite
163,165,11/9/2017,LAL,34,22,21,18,95,WAS,37,...,106.7,110.3,-3.14,100.9,104.2,107.8,Home,16,6.5,Favorite
37,39,10/22/2017,ATL,25,28,24,27,104,BRK,19,...,103.6,108.3,-4.73,97.2,106.4,111.2,Away,-12,-13.0,Underdog
